<a href="https://colab.research.google.com/github/mostly-sunny/digital-health-hackathon/blob/main/train_deep_coxph_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CoxPH Model을 활용한 유전자 별 치료 효능률 계산 

## Requirements

### Main Library
- pycox
- pytorch
- numpy
- pandas
- torchtuples

### Input File
- **all-in-one-modified.csv** (from eliminate_invalid_data.ipynb)
- **virtual-data.csv** (from make_virtual_data.ipynb)

In [ ]:
pip install pycox

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn_pandas import DataFrameMapper
import pandas as pd

import torch
import torchvision
import torchtuples as tt

from pycox.models import CoxPH
from pycox.evaluation import EvalSurv

import random

- Cuda (GPU) 가 사용 가능한지 확인

In [ ]:
torch.cuda.is_available()

True

# 재현성
- 코드의 재현성을 위한 랜덤 시드 고정
- 단, CPU에서 실행했을 때와 GPU에서 실행했을 때에는 결과가 달라지기 때문에, 동일하게 GPU에서만 학습을 진행

In [ ]:
random.seed(123456)
np.random.seed(123456)
torch.manual_seed(123456)

# 파일 설명

### all-in-one-modified.csv (from eliminate_invalid_data.ipynb) 
- 유전자 변이 유무, 임상 변수, 생존 기간, 사망 여부, 치료 유무를 가지고 있는 환자 데이터
- G1 ~ G300, Var1 ~ Var10, Treatment, time, event
- 주어진 조건 (0 <= 임상변수 <= 9)에 맞지 않는 임상변수 값을 가진 환자는 제거
- 생존기간인 time이 음수인 값을 가진 환자 또한 제거

### virtual-data.csv (from make_virtual_data.ipynb) 
- 치료 효능률을 계산하기 위한 데이터
- (0번째 행) : 유전자 변이 모두 0, 임상 변수 모두 0, 치료 0
- (1번째 행) : 유전자 변이 모두 0, 임상 변수 모두 0, 치료 1
- (2~301번째 행) : 유전자 변이 n-1에만 1, 임상 변수 모두 0, 치료 0
- (302~601번재 행) : 유전자 변이 n-301에만 1, 임상 변수 모두 0, 치료 1

In [ ]:
dataset = pd.read_csv('/content/all-in-one-modified.csv')
dataset_for_hr = pd.read_csv('/content/virtual-data.csv')

- all-in-one-modified.csv로부터 읽어들인 데이터 확인

In [ ]:
dataset.head()

,Unnamed: 0,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,G36,G37,G38,G39,...,G274,G275,G276,G277,G278,G279,G280,G281,G282,G283,G284,G285,G286,G287,G288,G289,G290,G291,G292,G293,G294,G295,G296,G297,G298,G299,G300,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10,Treatment,time,event
0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,5,1,1,4,6,5,2,1,0,1,0,57.448331,1
1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,3,9,1,1,2,4,0,1,0,27.004439,1
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,5,3,4,3,3,3,2,2,3,1,43.770511,1
3,3,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,2,7,2,3,5,0,1,4,5,3,1,32.281018,1
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,3,0,0,2,2,6,3,3,1,0,44.559284,0


# 데이터 비율 설정

### 테스트 데이터
- 전체 데이터(966개) 中 30% (290개)

### 훈련 데이터와 검증 데이터
- 전체 데이터(966개)에서 테스트 데이터(290개)를 뺀 (676개) 
- 그 中 훈련데이터 80% (541개) , 검증 데이터 20% (135개)

### 재현성
- 결과의 재현성을 위해 전체 데이터로부터 샘플링할 때 동일한 랜덤 시드를 사용하도록 함.

In [ ]:
dataset_test = dataset.sample(frac=0.3, random_state = np.random.seed(123456))
dataset_train = dataset.drop(dataset_test.index)
dataset_val = dataset_train.sample(frac=0.2, random_state = np.random.seed(123456))
dataset_train = dataset_train.drop(dataset_val.index)

# 데이터 전처리

### columns_standardize
- 임상변수 : 0~9 사이의 값을 MinMaxScaler로 Scaling

### columns_leave
- 유전자 변이 유무 + 치료 유무 :0과 1로 표현돼 있기 때문에 그대로 사용

### DataFrameMapper
- pandas.DataFrame에서 원하는 열을 뽑아서 리스트로 만들어주는 Object
- 리스트로 만들때 Scaler 함수가 포함된 열은 Scaler 적용
- None이면 값 변환 없이 그대로 적용

In [ ]:
columns_standardize = ['Var' + str(i) for i in range(1,11)]
columns_leave = ['G' + str(i) for i in range(1,301)]
columns_leave += ['Treatment']

standardize = [([col], MinMaxScaler()) for col in columns_standardize]

leave = [(col, None) for col in columns_leave]

x_mapper = DataFrameMapper(leave + standardize)

- 위에서 만든 DataFrameMapper로 DataFrame 중 입력 데이터를 모델이 학습할 수 있게 끔 리스트 형식으로 바꾸어 준다.

In [ ]:
x_train = x_mapper.fit_transform(dataset_train).astype('float32')
x_val = x_mapper.transform(dataset_val).astype('float32')
x_test = x_mapper.transform(dataset_test).astype('float32')
x_for_hr = x_mapper.transform(dataset_for_hr).astype('float32')

- DataFrame에서 출력 데이터인 time(생존시간)과 event(사망여부)를 뽑아 출력 데이터를 추린다.
- 검증(Validation)을 위한 입력, 출력 세트 val을 만든다.

In [ ]:
get_target = lambda df: (df['time'].values, df['event'].values)
y_train = get_target(dataset_train)
y_val = get_target(dataset_val)

durations_test, events_test = get_target(dataset_test)
val = x_val, y_val

# 딥러닝 네트워크 설계

### 함수 make_net
- pytorch network를 생성하여 반환하는 함수
- 매개변수 : input의 노드 수, output의 노드 수, 은닉층 수, 은닉층 노드 수, 은닉층 노드 감소 비율
- 활성화 함수로 ReLU 함수 사용
- 매 층마다 BatchNorm1d로 정규화
- 은닉층 수는 최소 1개부터 최대 4개까지 설정 가능.
- 은닉층 노드 감수 비율(rate)은 매 층마다 한 번씩 더 곱해짐.
- Dropout 비율은 0.2로 고정

예시) make_net(311, 1, 3, 1000, 0.5)
- 은닉층은 총 3층
- 첫번째 은닉층 노드 수 : 1000
- 두번째 은닉층 노드 수 : 500
- 세번째 은닉층 노드 수 : 250

In [ ]:
def make_net(in_features, out_features, hidden, nodes, rate):
  if hidden == 1:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.2),

      torch.nn.Linear(nodes, out_features)
    )
  elif hidden == 2:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.2),

      torch.nn.Linear(nodes, int(nodes * rate)),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(int(nodes * rate)),
      torch.nn.Dropout(0.2),
            
      torch.nn.Linear(int(nodes * rate), out_features)
    )
  elif hidden == 3:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.2),

      torch.nn.Linear(nodes, int(nodes * rate)),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(int(nodes * rate)),
      torch.nn.Dropout(0.2),

      torch.nn.Linear(int(nodes * rate), int(nodes * (rate ** 2))),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(int(nodes * (rate ** 2))),
      torch.nn.Dropout(0.2),
            
      torch.nn.Linear(int(nodes * (rate ** 2)), out_features)
    )
  elif hidden == 4:
    network =  torch.nn.Sequential(
      torch.nn.Linear(in_features, nodes),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(nodes),
      torch.nn.Dropout(0.2),

      torch.nn.Linear(nodes, int(nodes * rate)),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(int(nodes * rate)),
      torch.nn.Dropout(0.2),

      torch.nn.Linear(int(nodes * rate), int(nodes * (rate ** 2))),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(int(nodes * (rate ** 2))),
      torch.nn.Dropout(0.2),
      
      torch.nn.Linear(int(nodes * (rate ** 2)), int(nodes * (rate ** 3))),
      torch.nn.ReLU(),
      torch.nn.BatchNorm1d(int(nodes * (rate ** 3))),
      torch.nn.Dropout(0.2),

      torch.nn.Linear(int(nodes * (rate ** 3)), out_features)
    )
  return network

# 결과 저장을 위한 DataFrame
- 공통적으로 Network의 설정과 평가지표인 Brier score가 함께 저장된다.

### result_rank
- 치료와 상관 관계가 있는 유전자 변이의 순위를 저장하는 DataFrame

### result_treat_ratio
- 어떤 유전자 변이의 치료효능률을 저장하는 DataFrame

In [ ]:
columns_list = ['brier', 'layers', 'nodes', 'rate', 'lr']
for i in range(1, 301):
  columns_list.append('G' + str(i))
result_rank = pd.DataFrame(columns = columns_list)
result_treat_ratio = pd.DataFrame(columns = columns_list)

In [ ]:
result_treat_ratio

,brier,layers,nodes,rate,lr,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,...,G261,G262,G263,G264,G265,G266,G267,G268,G269,G270,G271,G272,G273,G274,G275,G276,G277,G278,G279,G280,G281,G282,G283,G284,G285,G286,G287,G288,G289,G290,G291,G292,G293,G294,G295,G296,G297,G298,G299,G300


In [ ]:
result_rank

,brier,layers,nodes,rate,lr,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,...,G261,G262,G263,G264,G265,G266,G267,G268,G269,G270,G271,G272,G273,G274,G275,G276,G277,G278,G279,G280,G281,G282,G283,G284,G285,G286,G287,G288,G289,G290,G291,G292,G293,G294,G295,G296,G297,G298,G299,G300


# 학습

### 변수 설명
- in_features : 입력데이터의 개수 (x_train.shape : 311 = 300(유전자) + 10(임상변수) + 1(치료유무))
- out_features : 출력노드의 개수
- layers_list : 은닉층 수를 가지고 있는 리스트
- nodes_list : 은닉층 노드 수를 가지고 있는 리스트
- rate_list : 은닉층 노드 감수 비율을 가지고 있는 리스트
- lr_list : 학습률을 가지고 있는 리스트
- total_num과 count : 학습 진행도를 확인하기 위한 변수

### 치료 효능률 계산 및 결과 저장
1. 훈련된 모델에 predict 함수를 통해 virtual-data.csv의 데이터를 입력
2. 각각의 가상 환자에 대한 위험비 도출
3. 300개 중 하나의 동일한 유전자 변이를 가졌으나 치료 유무만 다른 두 환자의 위험비의 비율 (치료효능률)을 2번으로부터 계산
4. 계산된 치료효능률을 result_treat_ratio의 저장
5. 치료효능률이 작은 순으로 순위를 매겨 각 유전자의 등수를 result_rank에 저장
6. 저장 시 해당 모델의 네트워크 설정과 Brier Score도 함께 저장

In [ ]:
in_features = x_train.shape[1]
out_features = 1

layers_list = [2]
nodes_list = [500]
rate_list = [(1/2), (1/3)]
lr_list = [0.001, 0]

total_num = len(layers_list) * len(nodes_list) * len(rate_list) * len(lr_list)
count = 1


for rate in rate_list:  
  for nodes in nodes_list:
    for layers in layers_list:
      for lr in lr_list:
        print(count, '/', total_num)
        print('layers:', layers, ', nodes:', nodes, ',rate:', rate, ',lr:', lr)
        count += 1

        # manage reproducibility
        random.seed(123456)
        np.random.seed(123456)
        torch.manual_seed(123456)

        # make model
        net = make_net(in_features, out_features, layers, nodes, rate)
        model = CoxPH(net, tt.optim.Adam)
        batch_size = len(dataset_train)

        # set learning rate
        if lr == 0:
          lrfinder = model.lr_finder(x_train, y_train, batch_size, tolerance = 10)
          model.optimizer.set_lr(lrfinder.get_best_lr())
        else:
          model.optimizer.set_lr(lr)
        
        epochs = 512
        callbacks = [tt.callbacks.EarlyStopping()]
        verbose = True

        # train
        %%time
        model.fit(x_train, y_train, batch_size, epochs, callbacks, verbose, val_data=val, val_batch_size=batch_size)
        
        # compute after train
        _ = model.compute_baseline_hazards()
        surv = model.predict_surv_df(x_test)

        # evaluation
        ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
        time_grid = np.linspace(durations_test.min(), durations_test.max(), 100)
        brier_score = ev.integrated_brier_score(time_grid)

        # calculate hazard ratio
        log_partial_hazard = model.predict(x_for_hr)
        partial_hazard = [np.exp(lph) for lph in log_partial_hazard]

        # ratio with treated and untreated
        treat_ratio = []
        for gene in range(300):
          treat_ratio.append([partial_hazard[gene+302]/partial_hazard[gene+2],'G' + str(gene+1)])
        
        treat_ratio_info = [float(i[0]) for i in treat_ratio]
        
        treat_ratio.sort()
        for i in range(300):
          treat_ratio[i][0] = i + 1
        treat_ratio.sort(key=lambda x:int(x[1][1:]))

        rank_info = [i[0] for i in treat_ratio]

        if lr == 0:
          info = [brier_score, layers, nodes, rate, lrfinder.get_best_lr()]
        else:
          info = [brier_score, layers, nodes, rate, lr]
        
        result_treat_ratio.loc[len(result_treat_ratio)] =  info + treat_ratio_info
        result_rank.loc[len(result_rank)] = info + rank_info

1 / 4
layers: 2 , nodes: 500 ,rate: 0.5 ,lr: 0.001
CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.34 µs
0:	[0s / 0s],		train_loss: 5.4996,	val_loss: 3.9214
1:	[0s / 0s],		train_loss: 5.2028,	val_loss: 3.9188
2:	[0s / 0s],		train_loss: 4.9943,	val_loss: 3.9158
3:	[0s / 0s],		train_loss: 4.8377,	val_loss: 3.9122
4:	[0s / 0s],		train_loss: 4.6982,	val_loss: 3.9078
5:	[0s / 0s],		train_loss: 4.5376,	val_loss: 3.9023
6:	[0s / 0s],		train_loss: 4.4251,	val_loss: 3.8954
7:	[0s / 0s],		train_loss: 4.3310,	val_loss: 3.8875
8:	[0s / 0s],		train_loss: 4.2166,	val_loss: 3.8781
9:	[0s / 0s],		train_loss: 4.1518,	val_loss: 3.8683
10:	[0s / 0s],		train_loss: 4.0838,	val_loss: 3.8580
11:	[0s / 0s],		train_loss: 4.0076,	val_loss: 3.8474
12:	[0s / 0s],		train_loss: 3.9486,	val_loss: 3.8363
13:	[0s / 0s],		train_loss: 3.8434,	val_loss: 3.8247
14:	[0s / 0s],		train_loss: 3.8393,	val_loss: 3.8124
15:	[0s / 0s],		train_loss: 3.8049,	val_loss: 3.8002
16:	[0s / 0s],		train_loss: 3.7902,	val_los

# 결과 저장

### ratio_은닉층수_노드수_노드수.csv
- 계산한 치료효능률을 저장한 DataFrame을 csv파일로 변환

### rank_은닉층수_노드수_노드수.csv
- 치료효능률이 낮은 순으로 등수를 매겼을 때의 등수를 저장한 DataFrame을 csv파일로 변환

In [ ]:
name_ratio = '/content/' + 'ratio_' + str(layers_list[0]) + '_' + str(nodes_list[0]) + '_' + str(nodes_list[-1]) + '.csv'
result_treat_ratio.to_csv(name_ratio)

In [ ]:
name_rank = '/content/' + 'rank_' + str(layers_list[0]) + '_' + str(nodes_list[0]) + '_' + str(nodes_list[-1]) + '.csv'
result_rank.to_csv(name_rank)

# 결과 확인
- Result DataFrame을 출력하여 결과 확인

In [ ]:
result_treat_ratio

,brier,layers,nodes,rate,lr,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,...,G261,G262,G263,G264,G265,G266,G267,G268,G269,G270,G271,G272,G273,G274,G275,G276,G277,G278,G279,G280,G281,G282,G283,G284,G285,G286,G287,G288,G289,G290,G291,G292,G293,G294,G295,G296,G297,G298,G299,G300
0,0.054480,2.0,500.0,0.500000,0.001000,0.625417,0.669693,0.615281,0.621247,0.619750,0.598648,0.597611,0.599472,0.645650,0.638257,0.603068,0.657343,0.587700,0.627347,0.599480,0.608166,0.643906,0.682481,0.596775,0.605929,0.632384,0.685813,0.643752,0.617804,0.640691,0.593315,0.630313,0.619031,0.602704,0.608979,0.574107,0.583281,0.684571,0.631347,0.555112,...,0.590253,0.611951,0.595669,0.636235,0.559861,0.631047,0.567723,0.635735,0.585110,0.609027,0.620222,0.613191,0.633759,0.598784,0.595109,0.645332,0.635577,0.607143,0.611502,0.571564,0.594072,0.642310,0.625451,0.621288,0.602620,0.659683,0.651923,0.610624,0.636150,0.636008,0.607970,0.576476,0.565698,0.631578,0.600524,0.642814,0.610750,0.644431,0.624060,0.650091
1,0.058183,2.0,500.0,0.500000,0.016681,0.963885,0.946839,0.950758,0.947686,0.963315,0.950659,0.957300,0.955720,0.960648,0.971301,0.941844,0.956339,0.962964,0.935173,0.947344,0.949802,0.953774,0.970140,0.951076,0.962465,0.962040,0.957202,0.958950,0.952093,0.965355,0.933968,0.952683,0.946246,0.955806,0.965100,0.948297,0.955130,0.955029,0.947536,0.945753,...,0.960760,0.947810,0.949631,0.954165,0.962013,0.951129,0.953876,0.954727,0.962996,0.944259,0.947829,0.967075,0.960128,0.939687,0.947569,0.964704,0.963108,0.944614,0.960009,0.947770,0.947218,0.958823,0.955434,0.953018,0.950028,0.952218,0.957738,0.960111,0.955910,0.946851,0.958310,0.954337,0.953094,0.954092,0.945365,0.953142,0.956181,0.966399,0.957250,0.939734
2,0.054357,2.0,500.0,0.333333,0.001000,0.637734,0.609998,0.582555,0.604622,0.625646,0.572599,0.561333,0.594257,0.641308,0.590662,0.550180,0.628146,0.579651,0.591832,0.525497,0.609653,0.622544,0.660299,0.588059,0.607341,0.637856,0.608778,0.589933,0.587245,0.606670,0.559299,0.596588,0.568148,0.566914,0.614406,0.589900,0.600456,0.585339,0.576154,0.537447,...,0.594278,0.561683,0.588809,0.590154,0.574527,0.613974,0.600926,0.650688,0.579551,0.610235,0.606388,0.639417,0.654659,0.567986,0.556418,0.605969,0.625031,0.608407,0.640125,0.565022,0.561374,0.587162,0.625853,0.576167,0.585427,0.566756,0.600988,0.631848,0.607474,0.642858,0.578135,0.577293,0.583146,0.609132,0.557392,0.602551,0.587133,0.619099,0.613527,0.608525
3,0.056771,2.0,500.0,0.333333,0.035112,0.792305,0.770919,0.764991,0.772378,0.777308,0.778354,0.773382,0.790169,0.763458,0.744303,0.780433,0.747921,0.774665,0.785436,0.785130,0.768612,0.729742,0.749597,0.770654,0.762143,0.795589,0.748751,0.751664,0.765398,0.757981,0.753867,0.748547,0.770304,0.772782,0.767527,0.784765,0.783713,0.763607,0.781699,0.803746,...,0.744058,0.761941,0.773496,0.777217,0.746857,0.772093,0.783546,0.776003,0.781627,0.755349,0.795388,0.766747,0.740122,0.802212,0.798678,0.744929,0.784237,0.749660,0.754514,0.760004,0.756420,0.743394,0.768410,0.790522,0.756620,0.770627,0.753628,0.797394,0.771944,0.759405,0.770791,0.770170,0.765816,0.804545,0.765866,0.772294,0.781619,0.735942,0.762181,0.746590


In [ ]:
result_rank

,brier,layers,nodes,rate,lr,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20,G21,G22,G23,G24,G25,G26,G27,G28,G29,G30,G31,G32,G33,G34,G35,...,G261,G262,G263,G264,G265,G266,G267,G268,G269,G270,G271,G272,G273,G274,G275,G276,G277,G278,G279,G280,G281,G282,G283,G284,G285,G286,G287,G288,G289,G290,G291,G292,G293,G294,G295,G296,G297,G298,G299,G300
0,0.054480,2.0,500.0,0.500000,0.001000,176.0,290.0,134.0,156.0,147.0,66.0,62.0,70.0,252.0,226.0,85.0,274.0,37.0,180.0,71.0,102.0,244.0,297.0,61.0,93.0,205.0,299.0,243.0,140.0,232.0,49.0,196.0,146.0,82.0,105.0,20.0,28.0,298.0,202.0,3.0,...,42.0,124.0,58.0,222.0,8.0,201.0,12.0,219.0,32.0,106.0,150.0,127.0,212.0,67.0,55.0,248.0,217.0,98.0,122.0,16.0,50.0,238.0,177.0,157.0,80.0,276.0,262.0,117.0,221.0,220.0,101.0,23.0,9.0,204.0,74.0,239.0,118.0,246.0,173.0,258.0
1,0.058183,2.0,500.0,0.500000,0.016681,272.0,53.0,96.0,66.0,269.0,95.0,196.0,171.0,244.0,298.0,18.0,180.0,265.0,4.0,59.0,86.0,143.0,296.0,103.0,260.0,259.0,193.0,219.0,113.0,281.0,3.0,123.0,48.0,172.0,279.0,76.0,161.0,158.0,61.0,40.0,...,248.0,70.0,83.0,149.0,258.0,106.0,145.0,154.0,266.0,29.0,71.0,291.0,237.0,9.0,63.0,276.0,267.0,33.0,234.0,68.0,57.0,218.0,167.0,129.0,89.0,116.0,206.0,236.0,174.0,54.0,213.0,152.0,132.0,148.0,36.0,133.0,176.0,287.0,194.0,10.0
2,0.054357,2.0,500.0,0.333333,0.001000,268.0,190.0,65.0,163.0,241.0,45.0,27.0,115.0,275.0,103.0,10.0,253.0,62.0,108.0,3.0,188.0,233.0,293.0,93.0,174.0,270.0,183.0,101.0,86.0,171.0,22.0,125.0,41.0,38.0,209.0,100.0,141.0,77.0,51.0,6.0,...,116.0,29.0,95.0,102.0,48.0,206.0,145.0,285.0,61.0,192.0,167.0,272.0,288.0,40.0,15.0,166.0,239.0,180.0,273.0,32.0,28.0,85.0,242.0,52.0,78.0,37.0,146.0,262.0,177.0,278.0,56.0,55.0,67.0,186.0,17.0,153.0,84.0,224.0,205.0,181.0
3,0.056771,2.0,500.0,0.333333,0.035112,272.0,174.0,130.0,180.0,209.0,216.0,188.0,260.0,123.0,21.0,225.0,34.0,193.0,252.0,251.0,155.0,1.0,40.0,171.0,111.0,282.0,38.0,46.0,133.0,82.0,59.0,36.0,167.0,184.0,149.0,246.0,243.0,124.0,235.0,294.0,...,19.0,110.0,189.0,208.0,31.0,178.0,242.0,201.0,234.0,67.0,281.0,143.0,12.0,292.0,289.0,24.0,245.0,41.0,61.0,97.0,75.0,17.0,153.0,263.0,77.0,170.0,56.0,287.0,177.0,93.0,172.0,165.0,135.0,295.0,137.0,179.0,233.0,5.0,113.0,29.0
